In [ ]:
import sys
sys.path.append('/home/hwang/anaconda3/lib/python3.7/site-packages/')

from sklearn import datasets
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import SimpleITK as sitk
import nibabel
import math
import texture

from keras.utils.np_utils import to_categorical
from keras import optimizers 
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, Input, Lambda, Reshape, AveragePooling2D, BatchNormalization
from keras.layers.merge import add, concatenate
from keras.applications.resnet50 import ResNet50, decode_predictions

from keras import backend as K
from keras.utils import plot_model

from keras.callbacks import EarlyStopping, LambdaCallback, ModelCheckpoint 
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization

from keras_unet.utils import plot_imgs, get_augmented, plot_segm_history
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance 
from keras_unet.models import custom_unet

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [ ]:
# delete .DS_Store file in each folder 

# resample
            
resampath = '/Users/hwang/Desktop/research/1_MM/Data/2_benign_MM/U-Net/pred/'
# resampath = '/Users/hwang/Documents/research/2_Prostate/Unet/roi_T2/Tz_resamp/train/'

resampfile = os.listdir(resampath)
resampfile.sort()
num_seg = len(resampfile)
count_seg = 0
for item_resamp in os.listdir(resampath):
    current_resamp = os.path.basename(item_resamp)
    if current_resamp == '.DS_Store':
        fullpath = os.path.join(resampath,current_resamp)
        os.remove(fullpath)

In [ ]:
# feature for deep learning (3D): img

resampath = '/Users/hwang/Desktop/research/1_MM/Data/2_benign_MM/resample_org_niigz/train/'
# resampath = '/Users/hwang/Documents/research/2_Prostate/Unet/nT2/Tz_resamp/train/'

resampfile = os.listdir(resampath)
resampfile.sort()
num_img = len(resampfile)

first = sitk.ReadImage(os.path.join(resampath,resampfile[0]))
feature = sitk.GetArrayFromImage(first)

feature_reshape = feature.transpose(1,2,0)

num = 1

for n in os.listdir(resampath):

    i = sitk.ReadImage(os.path.join(resampath,resampfile[num]))
    j = sitk.GetArrayFromImage(i)
    j_reshape = j.transpose(1,2,0)
    feature_reshape = np.dstack((feature_reshape, j_reshape))
    
    if num < num_img-1:
        num = num + 1  

In [ ]:
# feature for deep learning (3D): mask

roiresampath = '/Users/hwang/Desktop/research/1_MM/Data/2_benign_MM/resample_roi_niigz/train/'
# roiresampath = '/Users/hwang/Documents/research/2_Prostate/Unet/roi_T2/Tz_resamp/train/'

roiresampfile = os.listdir(roiresampath)
roiresampfile.sort()
num_seg = len(roiresampfile)

first = sitk.ReadImage(os.path.join(roiresampath,roiresampfile[0]))
mask = sitk.GetArrayFromImage(first) 
mask_reshape = mask.transpose(1,2,0)

num = 1

for n in os.listdir(resampath):

    i = sitk.ReadImage(os.path.join(roiresampath,roiresampfile[num]))
    j = sitk.GetArrayFromImage(i)
    j_reshape = j.transpose(1,2,0)
    mask_reshape = np.dstack((mask_reshape, j_reshape))
    
    if num < num_seg-1:
        num = num + 1  

In [ ]:
# feature normalization: 0-1

maxval = np.amax(feature)
feature_norm = feature_reshape/maxval

In [ ]:
X = feature_norm.copy()
XX = feature_norm.copy()
XXX = feature_norm.copy()

list_matrix = [X,XX,XXX]

y = np.stack(list_matrix,axis=3)
feature_reshape = y.transpose(2,0,1,3)
mask_reshape = mask_reshape.transpose(2,0,1)
mask_reshape = mask_reshape.reshape((mask_reshape.shape[0],mask_reshape.shape[1],mask_reshape.shape[2],1))

In [ ]:
X_train = feature_reshape
y_train = mask_reshape 

feature_reshape = []
mask_reshape = []

In [ ]:
train_gen = get_augmented(X_train, y_train, batch_size = 6, data_gen_args = dict(rotation_range=5., width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 40, zoom_range = 0.2, horizontal_flip = True, vertical_flip = True, fill_mode = 'constant'))
sample_batch = next(train_gen)
xx, yy = sample_batch
input_shape = X_train[0].shape

model = custom_unet(input_shape, filters = 32, use_batch_norm = True, dropout = 0.3, dropout_change_per_layer = 0.0, num_layers = 4)
model.summary()

In [ ]:
# feature for deep learning (3D): img

tresampath = '/Users/hwang/Desktop/research/1_MM/Data/2_benign_MM/resample_org_niigz/test/'
tresampfile = os.listdir(tresampath)
tresampfile.sort()
num_img = len(tresampfile)

first = sitk.ReadImage(os.path.join(tresampath,tresampfile[0]))
feature = sitk.GetArrayFromImage(first)

feature_reshape = feature.transpose(1,2,0)

num = 1

for n in os.listdir(tresampath):

    i = sitk.ReadImage(os.path.join(tresampath,tresampfile[num]))
    j = sitk.GetArrayFromImage(i)
    j_reshape = j.transpose(1,2,0)
    feature_reshape = np.dstack((feature_reshape, j_reshape))
    
    if num < num_img-1:
        num = num + 1 

In [ ]:
# feature for deep learning (3D): mask

mresampath = '/Users/hwang/Desktop/research/1_MM/Data/2_benign_MM/resample_roi_niigz/test/'
mresampfile = os.listdir(mresampath)
mresampfile.sort()
num_seg = len(mresampfile)

first = sitk.ReadImage(os.path.join(mresampath,mresampfile[0]))
mask = sitk.GetArrayFromImage(first) 
mask_reshape = mask.transpose(1,2,0)

num = 1

for n in os.listdir(mresampath):

    i = sitk.ReadImage(os.path.join(mresampath,mresampfile[num]))
    j = sitk.GetArrayFromImage(i)
    j_reshape = j.transpose(1,2,0)
    mask_reshape = np.dstack((mask_reshape, j_reshape))
    
    if num < num_seg-1:
        num = num + 1 

In [ ]:
# feature normalization: 0-1

maxval = np.amax(feature)
feature_norm = feature_reshape/maxval

In [ ]:
Y = feature_norm.copy()
YY = feature_norm.copy()
YYY = feature_norm.copy()

list_matrix = [Y,YY,YYY]

z = np.stack(list_matrix,axis=3)
feature_reshape = z.transpose(2,0,1,3)
mask_reshape = mask_reshape.transpose(2,0,1)
mask_reshape = mask_reshape.reshape((mask_reshape.shape[0],mask_reshape.shape[1],mask_reshape.shape[2],1))

In [ ]:
X_test = feature_reshape
y_test = mask_reshape 

feature_reshape = []
mask_reshape = []

In [ ]:
im = plt.imshow(X_test[865,:,:,0])

In [ ]:
im = plt.imshow(y_test[788,:,:,0])

In [ ]:
model_filename = 'segm_model2_v3.h5'
# es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1)
callback_checkpoint = ModelCheckpoint(model_filename,verbose = 1, monitor = 'val_loss', save_best_only=True)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = [iou, iou_thresholded])
history = model.fit_generator(train_gen, steps_per_epoch = 200, epochs = 50, validation_data = (X_test, y_test), callbacks = [callback_checkpoint])

In [ ]:
model.load_weights(model_filename)
y_pred = model.predict(X_test)

In [ ]:
plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=19)

In [ ]:
def dice_coef2(y_test, y_pred):
    y_test_f = y_test.flatten()
    y_pred_f = y_pred.flatten()
    union = np.sum(y_test_f) + np.sum(y_pred_f)
    if union==0: return 1
    intersection = np.sum(y_test_f * y_pred_f)
    return 2. * intersection / union

In [ ]:
dice_coef2(y_test, y_pred)

In [ ]:
image = y_pred
# image = image[:,:,:,1]
image2 = image.reshape(image.shape[0],image.shape[1],image.shape[2])

In [ ]:
# save U-Net segmented BM 

predpath = '/Users/hwang/Desktop/research/1_MM/Data/2_benign_MM/U-Net/pred/'

mresampath = '/Users/hwang/Desktop/research/1_MM/Data/2_benign_MM/resample_roi_niigz/test/'
mresampfile = os.listdir(mresampath)
mresampfile.sort()
num_img = len(mresampfile)

num = 0

for n in os.listdir(mresampath):

    numslice = 19
    fslice = num*numslice
    lslice = ((num+1)*numslice)-1
    image3 = image2[fslice:lslice,:,:]
    image4 = sitk.GetImageFromArray(image3)
    image4.SetSpacing([0.5303, 0.5303, 3])
    sitk.WriteImage(image4,os.path.join(predpath,mresampfile[num]),True)
    
    if num < num_img-1:
        num = num + 1